In [1]:
import torch
from src.net.model import ViTSTRTransducer

In [ ]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
checkpoint_path = "/home/vadim/DL/work/ViTSTR/weights/ViTSTRTransducer-BEST.ckpt"
model = ViTSTRTransducer.load_from_checkpoint(checkpoint_path, training=False, map_location=DEVICE).eval()
model.freeze()

Successfully loaded ViTSTR-T weights!



In [3]:
model_ts = model.half().to_torchscript('ViTSTR-T-half.torchscript')
params = {i: param.dtype == torch.half for i, param in enumerate(model_ts.parameters())}
print(f'Total params: {len(params.keys())}\nTotal FP16 params: {sum(params.values())}')

Total params: 175
Total FP16 params: 175


In [4]:
# Image input shape: [1, C, H, W]
img_input = torch.rand(1, 3, 224, 224, device=DEVICE).half()
# Sequence length
seq_len = 17
vocab_size = 17
target_input = torch.randint(0, vocab_size, (1, seq_len), dtype=torch.int32, device=DEVICE)
# Output shape: [1, seq, vocab_size]
output = model_ts(img_input, target_input)
output.shape

torch.Size([1, 17, 17])